In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
from backtesting.backtest import Backtest
from backtesting.scenarios import Scenario
from example.config import DEFAULT_CONSTRAINTS, DEFAULT_PORTFOLIO_SETUP, DEFAULT_STRATEGY_CONFIG
from data.data import Benchmarks


scenario = Scenario(
    name="test backtest 10y",
    start_date="2020-01-01",
    end_date="2025-05-01",
    benchmark=Benchmarks.SP500,
    constraints=DEFAULT_CONSTRAINTS,
    portfolio_config=DEFAULT_PORTFOLIO_SETUP,
    verbose=True,
    portfolio_name="sp500_10y_long_only_macd_zscore",
)  
scenario.set_strategies(DEFAULT_STRATEGY_CONFIG)
backtest = Backtest(scenario, verbose=True)


Portfolio setup: {
    "initial_capital": 100000,
    "initial_holdings": {},
    "initial_value": 0,
    "capital_growth_freq": "M",
    "new_capital_growth_pct": 0,
    "new_capital_growth_amt": 10000,
    "allocation_method": "equal",
    "excluded_sectors": [],
    "included_countries": [
        "United States"
    ],
    "min_market_cap": 0,
    "max_market_cap": Infinity,
    "trailing_stop_loss_pct": 0.05,
    "trailing_update_threshold": 0.02
}
Strategies: {
    "macd_x": true,
    "z": true
}


In [4]:
backtest.run_batch(verbose=True)

Backtest starting... swoosh!
Universe size: 480
Total trading days: 1340
Total strategies: 2
Starting in 2020-01-01
Ending in 2025-05-01


Backtesting by strategy: 100%|██████████| 2/2 [00:05<00:00,  2.75s/strategy]


Backtest completed!


In [30]:
# backtest.generate_report(filename="sample_report")

In [5]:
p = backtest.get_portfolio()

In [19]:
a = backtest.generate_advanced_analytics()

In [ ]:
from portfolio.portfolio import ExitReason


ticker_metrics = []
for ticker, trades in a.ticker_level_records_ts.items():
    df = pd.DataFrame(trades).T
    ticker_metrics.append({
        "ticker": ticker,
        "buy": df['total_long_trades'].sum(),
        "sell": len(df),
        "duration": df["holding_period"].mean(),
        "return": df["return"].mean(),
        "profit": df["profit"].mean(),
        "stop_loss_count": df[df["exit_reason"] == ExitReason.STOP_LOSS].shape[0],
    })


2020-01-03    1
2020-07-16    1
2020-10-02    1
2021-01-15    1
2021-03-15    1
2021-03-29    1
2021-05-12    1
2021-05-25    1
2021-08-16    1
2021-09-27    1
2021-10-15    1
2021-11-17    1
2021-11-22    1
2021-12-16    1
2021-12-20    1
2022-01-07    1
2022-02-28    1
2022-04-12    1
2022-05-20    1
2023-03-24    1
2023-07-03    1
2023-08-24    1
2023-08-28    1
2023-09-21    1
2023-11-06    1
2024-04-15    1
2024-05-07    1
2024-05-16    1
2024-05-23    1
2024-07-17    1
2025-02-21    1
2025-02-27    1
Name: total_long_trades, dtype: object
2020-01-03    1
2020-02-26    1
2020-05-11    1
2020-05-21    1
2020-08-25    1
2020-10-23    1
2020-12-14    1
2021-03-09    1
2021-03-25    1
2021-05-18    1
2021-07-16    1
2021-09-03    1
2021-11-30    1
2021-12-08    1
2021-12-28    1
2022-01-18    1
2022-02-16    1
2022-03-03    1
2022-03-17    1
2022-04-22    1
2022-05-05    1
2022-08-30    1
2023-01-26    1
2023-03-03    1
2023-03-13    1
2023-09-12    1
2024-06-14    1
2024-07-31    1
2

In [29]:
signal_history =p.signals_history


In [26]:
no_signal_days = sum(1 for date, signals in signal_history.items() if not signals)

,holding_start_date,holding_end_date,holding_period,cost_basis,total_long_trades,profit,return,exit_reason
2024-07-15,2024-07-10 00:00:00,2024-07-15 00:00:00,5,48.099998,1,25.053005,0.014137,ExitReason.SELL
2024-12-18,2024-12-06 00:00:00,2024-12-18 00:00:00,12,69.510002,1,-32.417676,-0.01525,ExitReason.SELL
2025-02-21,2025-01-15 00:00:00,2025-02-21 00:00:00,37,73.072171,1,130.556322,0.019336,ExitReason.SELL


In [38]:
sum([(np.array(signals.values()) == 0).all() for signals in a.portfolio.signals_history.values()])

np.int64(0)